In [ ]:
from dotenv import load_dotenv
import os
from pathlib import Path

CURRENT_DIRECTORY_NOTEBOOK = None


def intitate_notebook():
    load_dotenv()
    global CURRENT_DIRECTORY_NOTEBOOK
    if CURRENT_DIRECTORY_NOTEBOOK is None:
        os.chdir(os.getenv("PROJECT_BASE_PATH"))
        CURRENT_DIRECTORY_NOTEBOOK = Path(os.getcwd())
        print("Current directory for notebook: ", CURRENT_DIRECTORY_NOTEBOOK)
    else:
        print(
            "Current directory for notebook is already set: ",
            CURRENT_DIRECTORY_NOTEBOOK,
        )


intitate_notebook()

#### Feature Engineering - Code
O - https://chatgpt.com/c/683b0134-e464-800c-b614-ccc268735823

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import DataFrame
from pyspark.sql.functions import (
    count,
    sum as Fsum,
    avg,
    max as Fmax,
    min as Fmin,
    stddev,
    broadcast,
    when,
    col,
)

In [ ]:
spark = (
    SparkSession.builder.appName("PostgresETL")
    .config("spark.jars", "setup_files/postgresql-42.7.5.jar")
    .config("spark.driver.memory", "4g")
    .config("spark.executor.memory", "2g")
    .config("spark.sql.shuffle.partitions", "100")
    .getOrCreate()
)


username = "data_source_user"
password = "data_source_user_password"
host = "172.17.0.1"
port = "5435"
database = "data_source_db"


jdbc_url = f"jdbc:postgresql://{host}:{port}/{database}"
properties = {"user": username, "password": password, "driver": "org.postgresql.Driver"}

In [ ]:
bureau_df = spark.read.option("failFast", "true").jdbc(
    url=jdbc_url, table="bureau", properties=properties
)

bureau_balance_df = spark.read.option("failFast", "true").jdbc(
    url=jdbc_url, table="bureau_balance", properties=properties
)

In [ ]:
def bureau_balance_features(
    bureau_balance: DataFrame, bureau: DataFrame, target_shuffle_partitions: int = None
) -> DataFrame:
    """
    5. BUREAU BALANCE FEATURES
    Join bureau_balance with bureau to get sk_id_curr,
    then aggregate bureau_balance features grouped by sk_id_curr.

    1) Project-down to only the columns we need from both tables.
    2) Broadcast 'bureau' (which is usually much smaller) to avoid a full shuffle join.
    3) Repartition by 'sk_id_curr' before doing the groupBy so that the per-group shuffle is minimized.
    4) Optionally, reduce spark.sql.shuffle.partitions if your cluster is small.

    Args:
      bureau_balance: DataFrame with columns [sk_id_bureau, months_balance, status, ...other columns...]
      bureau:         DataFrame with columns [sk_id_bureau, sk_id_curr, ...other columns...]
      target_shuffle_partitions: if set, we will override spark.sql.shuffle.partitions to this value.

    Returns:
      DataFrame with one row per sk_id_curr and all the aggregated bureau_balance features.
    """

    # -------------------------------------------------------------------------
    # 1) Project only the needed columns from each DataFrame
    # -------------------------------------------------------------------------
    bb_small = bureau_balance.select("sk_id_bureau", "months_balance", "status")
    bureau_balance = bureau_balance.repartition("sk_id_bureau")
    b_small = bureau.select("sk_id_bureau", "sk_id_curr")

    # -------------------------------------------------------------------------
    # 2) Broadcast-join: if 'bureau' is much smaller than 'bureau_balance',
    #    we avoid a large shuffle on sk_id_bureau
    # -------------------------------------------------------------------------
    joined = bb_small.join(broadcast(b_small), on="sk_id_bureau", how="left")

    # -------------------------------------------------------------------------
    # 3) Repartition by the grouping key to keep each sk_id_curr on one partition
    # -------------------------------------------------------------------------
    repartitioned = joined.repartition("sk_id_curr")

    # -------------------------------------------------------------------------
    # 4) Define all of our aggregate expressions in one go.
    #    We compute basic counts, min/max of months_balance,
    #    status-counts, DPD‐related counts, and 12‐month‐window counts.
    # -------------------------------------------------------------------------
    agg_exprs = [
        # Basic counts and min/max:
        F.count("months_balance").alias("bureau_balance_months_count"),
        F.min("months_balance").alias("bureau_balance_months_min"),
        F.max("months_balance").alias("bureau_balance_months_max"),
        (F.max("months_balance") - F.min("months_balance")).alias(
            "bureau_balance_history_length"
        ),
        # Status‐by‐status counts (0,1,2,3,4,5,C,X):
        F.sum(when(col("status") == "0", 1).otherwise(0)).alias(
            "bureau_status_0_count"
        ),
        F.sum(when(col("status") == "1", 1).otherwise(0)).alias(
            "bureau_status_1_count"
        ),
        F.sum(when(col("status") == "2", 1).otherwise(0)).alias(
            "bureau_status_2_count"
        ),
        F.sum(when(col("status") == "3", 1).otherwise(0)).alias(
            "bureau_status_3_count"
        ),
        F.sum(when(col("status") == "4", 1).otherwise(0)).alias(
            "bureau_status_4_count"
        ),
        F.sum(when(col("status") == "5", 1).otherwise(0)).alias(
            "bureau_status_5_count"
        ),
        F.sum(when(col("status") == "C", 1).otherwise(0)).alias(
            "bureau_status_c_count"
        ),
        F.sum(when(col("status") == "X", 1).otherwise(0)).alias(
            "bureau_status_x_count"
        ),
        # Total number of status records (for ratio denominator)
        F.count("status").alias("bureau_status_total_count"),
        # DPD‐related counts (DPD = days past due)
        F.sum(when(col("status").isin("1", "2", "3", "4", "5"), 1).otherwise(0)).alias(
            "bureau_dpd_count"
        ),
        F.sum(when(col("status").isin("2", "3", "4", "5"), 1).otherwise(0)).alias(
            "bureau_dpd_30_count"
        ),
        F.sum(when(col("status").isin("3", "4", "5"), 1).otherwise(0)).alias(
            "bureau_dpd_60_count"
        ),
        F.sum(when(col("status").isin("4", "5"), 1).otherwise(0)).alias(
            "bureau_dpd_90_count"
        ),
        F.sum(when(col("status") == "5", 1).otherwise(0)).alias(
            "bureau_severe_dpd_count"
        ),
        # “Recent 12 months” behavior (months_balance >= -12):
        F.sum(
            when(
                (col("months_balance") >= -12)
                & col("status").isin("1", "2", "3", "4", "5"),
                1,
            ).otherwise(0)
        ).alias("bureau_recent_dpd_count_12m"),
        F.sum(
            when((col("months_balance") >= -12) & (col("status") == "5"), 1).otherwise(
                0
            )
        ).alias("bureau_recent_severe_dpd_12m"),
        F.sum(
            when((col("months_balance") >= -12) & (col("status") == "C"), 1).otherwise(
                0
            )
        ).alias("bureau_recent_closed_12m"),
    ]

    # -------------------------------------------------------------------------
    # 5) Perform the groupBy‐aggregation
    # -------------------------------------------------------------------------
    grouped = repartitioned.groupBy("sk_id_curr").agg(*agg_exprs)

    # -------------------------------------------------------------------------
    # 6) Finally, compute the ratio columns.  We do null‐safe division in case
    #    bureau_status_total_count is zero (we cast to double by dividing by a literal 1.0).
    # -------------------------------------------------------------------------
    # You could also use:
    #   F.when(col("bureau_status_total_count") > 0,
    #          col("bureau_dpd_count") / col("bureau_status_total_count")
    #   ).otherwise(0.0)
    # if you prefer to explicitly handle divide‐by‐zero.
    #
    results = (
        grouped.withColumn(
            "bureau_dpd_ratio",
            col("bureau_dpd_count") / (col("bureau_status_total_count") + F.lit(0.0)),
        )
        .withColumn(
            "bureau_dpd_30_ratio",
            col("bureau_dpd_30_count")
            / (col("bureau_status_total_count") + F.lit(0.0)),
        )
        .withColumn(
            "bureau_dpd_60_ratio",
            col("bureau_dpd_60_count")
            / (col("bureau_status_total_count") + F.lit(0.0)),
        )
        .withColumn(
            "bureau_dpd_90_ratio",
            col("bureau_dpd_90_count")
            / (col("bureau_status_total_count") + F.lit(0.0)),
        )
        .withColumn(
            "bureau_severe_dpd_ratio",
            col("bureau_severe_dpd_count")
            / (col("bureau_status_total_count") + F.lit(0.0)),
        )
    )

    return results

In [ ]:
def bureau_basic_counts_features(bureau: DataFrame) -> DataFrame:
    """
    1. BASIC AGGREGATION FEATURES: Credit counts and credit type distribution
    """
    credit_active_counts = [
        ("active", "bureau_active_count"),
        ("closed", "bureau_closed_count"),
        ("sold", "bureau_sold_count"),
        ("bad debt", "bureau_bad_debt_count"),
    ]
    credit_type_counts = [
        ("Consumer credit", "bureau_consumer_loan_count"),
        ("Credit card", "bureau_credit_card_count"),
        ("Mortgage", "bureau_mortgage_count"),
        ("Car loan", "bureau_car_loan_count"),
        ("Microloan", "bureau_microloan_count"),
    ]

    agg_exprs = [count("sk_id_bureau").alias("bureau_credit_count")]

    # Add credit_active conditional counts
    for val, alias_name in credit_active_counts:
        agg_exprs.append(
            Fsum(when(col("credit_active") == val, 1).otherwise(0)).alias(alias_name)
        )
    # Add credit_type conditional counts
    for val, alias_name in credit_type_counts:
        agg_exprs.append(
            Fsum(when(col("credit_type") == val, 1).otherwise(0)).alias(alias_name)
        )

    return bureau.groupBy("sk_id_curr").agg(*agg_exprs)


def bureau_amount_based_features(bureau: DataFrame) -> DataFrame:
    """
    2. AMOUNT-BASED FEATURES: sums, means, ratios on credit, debt, limits, overdue
    """
    df = bureau

    # Aggregations on AMT_CREDIT_SUM and related
    agg_exprs = [
        Fsum("amt_credit_sum").alias("bureau_amt_credit_sum_total"),
        avg("amt_credit_sum").alias("bureau_amt_credit_sum_mean"),
        Fmax("amt_credit_sum").alias("bureau_amt_credit_sum_max"),
        Fmin("amt_credit_sum").alias("bureau_amt_credit_sum_min"),
        stddev("amt_credit_sum").alias("bureau_amt_credit_sum_std"),
        Fsum("amt_credit_sum_debt").alias("bureau_amt_credit_sum_debt_total"),
        avg("amt_credit_sum_debt").alias("bureau_amt_credit_sum_debt_mean"),
        Fmax("amt_credit_sum_debt").alias("bureau_amt_credit_sum_debt_max"),
        # Debt to credit ratio (safe divide)
        (Fsum("amt_credit_sum_debt") / Fsum("amt_credit_sum")).alias(
            "bureau_debt_credit_ratio"
        ),
        Fsum("amt_credit_sum_limit").alias("bureau_amt_credit_sum_limit_total"),
        avg("amt_credit_sum_limit").alias("bureau_amt_credit_sum_limit_mean"),
        # Credit utilization ratio (debt / limit)
        (Fsum("amt_credit_sum_debt") / Fsum("amt_credit_sum_limit")).alias(
            "bureau_limit_utilization_ratio"
        ),
        Fsum("amt_credit_sum_overdue").alias("bureau_amt_credit_sum_overdue_total"),
        avg("amt_credit_sum_overdue").alias("bureau_amt_credit_sum_overdue_mean"),
        Fmax("amt_credit_max_overdue").alias("bureau_amt_credit_max_overdue_max"),
        # Has overdue flag: max of indicator if overdue > 0
        Fmax(when(col("amt_credit_sum_overdue") > 0, 1).otherwise(0)).alias(
            "bureau_has_overdue_flag"
        ),
    ]

    return df.groupBy("sk_id_curr").agg(*agg_exprs)


def bureau_time_based_features(bureau: DataFrame) -> DataFrame:
    """
    3. TIME-BASED FEATURES: credit age, end date, update recency, recent credit counts
    """
    df = bureau

    # Recent credit counts in last 1 and 2 years (-365 and -730 days)
    recent_1y = Fsum(when(col("days_credit") >= -365, 1).otherwise(0)).alias(
        "bureau_recent_credit_count_1y"
    )
    recent_2y = Fsum(when(col("days_credit") >= -730, 1).otherwise(0)).alias(
        "bureau_recent_credit_count_2y"
    )

    agg_exprs = [
        Fmin("days_credit").alias("bureau_days_credit_min"),
        Fmax("days_credit").alias("bureau_days_credit_max"),
        avg("days_credit").alias("bureau_days_credit_mean"),
        (Fmax("days_credit") - Fmin("days_credit")).alias(
            "bureau_credit_history_length"
        ),
        recent_1y,
        recent_2y,
        Fmin("days_credit_enddate").alias("bureau_days_credit_enddate_min"),
        Fmax("days_credit_enddate").alias("bureau_days_credit_enddate_max"),
        avg("days_credit_enddate").alias("bureau_days_credit_enddate_mean"),
        Fmin("days_credit_update").alias("bureau_days_credit_update_min"),
        Fmax("days_credit_update").alias("bureau_days_credit_update_max"),
        avg("days_credit_update").alias("bureau_days_credit_update_mean"),
    ]

    return df.groupBy("sk_id_curr").agg(*agg_exprs)


def bureau_delinquency_features(bureau: DataFrame) -> DataFrame:
    """
    4. DELINQUENCY FEATURES: days overdue, dpd counts, credit prolongations
    """
    df = bureau

    agg_exprs = [
        Fmax("credit_day_overdue").alias("bureau_credit_day_overdue_max"),
        avg("credit_day_overdue").alias("bureau_credit_day_overdue_mean"),
        Fmax(when(col("credit_day_overdue") > 0, 1).otherwise(0)).alias(
            "bureau_has_dpd_flag"
        ),
        Fsum(when(col("credit_day_overdue") >= 30, 1).otherwise(0)).alias(
            "bureau_dpd_30_count"
        ),
        Fsum(when(col("credit_day_overdue") >= 60, 1).otherwise(0)).alias(
            "bureau_dpd_60_count"
        ),
        Fsum(when(col("credit_day_overdue") >= 90, 1).otherwise(0)).alias(
            "bureau_dpd_90_count"
        ),
        Fsum("cnt_credit_prolong").alias("bureau_cnt_credit_prolong_sum"),
        Fmax("cnt_credit_prolong").alias("bureau_cnt_credit_prolong_max"),
        avg("cnt_credit_prolong").alias("bureau_cnt_credit_prolong_mean"),
        Fmax(when(col("cnt_credit_prolong") > 0, 1).otherwise(0)).alias(
            "bureau_has_prolongation_flag"
        ),
    ]

    return df.groupBy("sk_id_curr").agg(*agg_exprs)


def bureau_currency_and_annuity_features(bureau: DataFrame) -> DataFrame:
    """
    7. CURRENCY AND ANNUITY FEATURES
    """
    df = bureau

    agg_exprs = [
        F.countDistinct("credit_currency").alias("bureau_currency_variety"),
        Fmax(when(col("credit_currency") != "currency 1", 1).otherwise(0)).alias(
            "bureau_foreign_currency_flag"
        ),
        Fsum("amt_annuity").alias("bureau_amt_annuity_total"),
        avg("amt_annuity").alias("bureau_amt_annuity_mean"),
        Fmax("amt_annuity").alias("bureau_amt_annuity_max"),
    ]

    return df.groupBy("sk_id_curr").agg(*agg_exprs)


def bureau_cross_table_ratio_features(bureau: DataFrame) -> DataFrame:
    """
    8. CROSS-TABLE RATIO FEATURES: Utilization ratios by credit type and active/closed debt ratios
    """
    df = bureau

    # Group by sk_id_curr and credit_type for sums needed
    agg_by_type = df.groupBy("sk_id_curr", "credit_type").agg(
        Fsum("amt_credit_sum_debt").alias("sum_debt"),
        Fsum("amt_credit_sum_limit").alias("sum_limit"),
        Fsum("amt_credit_sum").alias("sum_credit"),
        Fsum(
            when(
                col("credit_active") == "Active", col("amt_credit_sum_debt")
            ).otherwise(0)
        ).alias("active_debt"),
    )

    # Pivot utilization ratios for credit card and mortgage types
    credit_card = agg_by_type.filter(col("credit_type") == "Credit card").select(
        "sk_id_curr",
        (col("sum_debt") / col("sum_limit")).alias("bureau_cc_utilization_ratio"),
    )

    mortgage = agg_by_type.filter(col("credit_type") == "Mortgage").select(
        "sk_id_curr",
        (col("sum_debt") / col("sum_credit")).alias("bureau_mortgage_utilization"),
    )

    # Sum total debt and active debt per sk_id_curr for active debt ratio
    debt_totals = (
        df.groupBy("sk_id_curr")
        .agg(
            Fsum("amt_credit_sum_debt").alias("total_debt"),
            Fsum(
                when(
                    col("credit_active") == "Active", col("amt_credit_sum_debt")
                ).otherwise(0)
            ).alias("active_debt_sum"),
        )
        .withColumn(
            "bureau_active_debt_ratio", col("active_debt_sum") / col("total_debt")
        )
    )

    # Closed early ratio: count closed early credits / total credits
    closed_early = (
        df.withColumn(
            "closed_early_flag",
            when(
                (col("credit_active") == "Closed")
                & (col("days_enddate_fact") < col("days_credit_enddate")),
                1,
            ).otherwise(0),
        )
        .groupBy("sk_id_curr")
        .agg(
            (Fsum("closed_early_flag") / count("sk_id_bureau")).alias(
                "bureau_closed_early_ratio"
            )
        )
    )

    # Join all cross-table ratios
    from functools import reduce

    dfs = [
        credit_card,
        mortgage,
        debt_totals.select("sk_id_curr", "bureau_active_debt_ratio"),
        closed_early,
    ]

    df_final = reduce(
        lambda left, right: left.join(right, on="sk_id_curr", how="outer"), dfs
    )

    return df_final.fillna(0)  # Fill nulls from joins with 0

In [ ]:
basic_counts_df = bureau_basic_counts_features(bureau_df)
amount_based_df = bureau_amount_based_features(bureau_df)
time_based_df = bureau_time_based_features(bureau_df)
delinquency_df = bureau_delinquency_features(bureau_df)
currency_annuity_df = bureau_currency_and_annuity_features(bureau_df)
balance_df = bureau_balance_features(bureau_balance_df, bureau_df)
cross_table_df = bureau_cross_table_ratio_features(bureau_df)

final_df = (
    basic_counts_df.join(amount_based_df, "sk_id_curr", "left")
    .join(time_based_df, "sk_id_curr", "left")
    .join(delinquency_df, "sk_id_curr", "left")
    .join(currency_annuity_df, "sk_id_curr", "left")
    .join(balance_df, "sk_id_curr", "left")
    .join(cross_table_df, "sk_id_curr", "left")
)

final_df.show()